### 数据处理

In [55]:
import pandas as pd
import numpy as np

In [57]:
# file_Countries = 'ProcessedData/Countries.csv'
# file_Athletes = 'ProcessedData/Athletes.csv'
# file_Medals = 'ProcessedData/Medals.csv'
file_countries = 'Countries.csv'
file_athletes = 'data/summerOly_athletes.csv'
file_medals = 'Medals.csv'
file_programs = 'data/summerOly_programs.csv'

df_countries = pd.read_csv(
    file_countries,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
print(df_countries)
df_athletes = pd.read_csv(
    file_athletes,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
df_athletes.loc[df_athletes["NOC"]=="ROC", "NOC"] = "RUS"
df_athletes.loc[df_athletes["NOC"]=="LBN", "NOC"] = "LBI"
print(df_athletes)
df_medals = pd.read_csv(
    file_medals,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
print(df_medals)
df_programs = pd.read_csv(
    file_programs,
    encoding='utf-8',
    encoding_errors='ignore'  # 忽略解码错误
)
print(df_programs)

     Unnamed: 0  NOC                      Country_Name          Host_Years
0             0  AFG                       Afghanistan                 NaN
1             1  AHO              Netherlands Antilles                 NaN
2             2  AIN                               AIN                 NaN
3             3  ALB                           Albania                 NaN
4             4  ALG                           Algeria                 NaN
5             5  AND                           Andorra                 NaN
6             6  ANG                            Angola                 NaN
7             7  ANT               Antigua and Barbuda                 NaN
8             8  ANZ                       Australasia                 NaN
9             9  ARG                         Argentina                 NaN
10           10  ARM                           Armenia                 NaN
11           11  ARU                             Aruba                 NaN
12           12  ASA     

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [4]:
# # 1. 按 NOC、Year 和 Sport 分组
# grouped = df_athletes.groupby(['NOC', 'Year', 'Sport'])
# print(grouped)

# # 2. 统计每个组的参赛人数    TODO:考虑重名？
# contestants = grouped[['Name', 'Sex']].nunique().reset_index()
# contestants = contestants.rename(columns={'Name': 'Participants'})
# print(contestants)

# # 3. 统计男女比例
# gender_ratio = grouped['Sex'].apply(lambda x: x['Name'].nunique()).unstack(fill_value=0).reset_index()
# print(gender_ratio.columns)
# gender_ratio.columns = ['NOC', 'Year', 'Sport', 'Female', 'Male']
# print(gender_ratio)

# 1. 按 NOC、Year 和 Sport 分组
grouped = df_athletes.groupby(['NOC', 'Year', 'Sport'])
print(grouped)

# 2. 统计每个组的参赛人数（去重，考虑重名情况）
df_unique = df_athletes.drop_duplicates(subset=['Name', 'NOC', 'Year', 'Sport'])
contestants = df_unique.groupby(['NOC', 'Year', 'Sport'])[['Name', 'Sex']].nunique().reset_index()
contestants = contestants.rename(columns={'Name': 'Participants'})
print(contestants)

# 3. 统计每个组的男女人数比
gender_count = df_unique.groupby(['NOC', 'Year', 'Sport', 'Sex']).size().unstack(fill_value=0).reset_index()

# 4. 合并参赛人数与男女人数数据
gender_ratio = pd.merge(contestants, gender_count, on=['NOC', 'Year', 'Sport'], how='left')

# 5. 给列命名并显示结果
gender_ratio = gender_ratio[['NOC', 'Year', 'Sport', 'Participants', 'F', 'M']]
gender_ratio.columns = ['NOC', 'Year', 'Sport', 'Participants', 'Female', 'Male']
print(gender_ratio)

# 4. 统计得奖情况，创建一个新的列，标记是否得奖
df_athletes['Medal_Status'] = df_athletes['Medal'].apply(lambda x: 'No medal' if pd.isna(x) else x)
print(df_athletes)

# 统计金、银、铜和无奖的数量
medals = df_athletes.groupby(['NOC', 'Year', 'Sport', 'Medal_Status']).size().unstack(fill_value=0).reset_index()
print(medals)

# 5. 将参赛人数、男女比和得奖情况合并成一个最终的 DataFrame
summary = gender_ratio
summary = pd.merge(summary, medals, on=['NOC', 'Year', 'Sport'], how='left')
print(summary)

# 6. 如果需要，对每个运动项目的得奖人数进行进一步处理
summary['Gold'] = summary.get('Gold', 0)
summary['Silver'] = summary.get('Silver', 0)
summary['Bronze'] = summary.get('Bronze', 0)
summary['No medal'] = summary.get('No medal', 0)

# 显示最终汇总表
print(summary)
summary.to_csv("sports_summary.csv")

       NOC  Year      Sport  Participants  Sex
0      AFG  1936  Athletics             2    1
1      AFG  1936     Hockey            13    1
2      AFG  1948   Football            10    1
3      AFG  1948     Hockey            12    1
4      AFG  1956     Hockey            12    1
...    ...   ...        ...           ...  ...
26224  ZIM  2020     Rowing             1    1
26225  ZIM  2020   Swimming             2    2
26226  ZIM  2024  Athletics             4    2
26227  ZIM  2024     Rowing             1    1
26228  ZIM  2024   Swimming             2    2

[26229 rows x 5 columns]
       NOC  Year      Sport  Participants  Female  Male
0      AFG  1936  Athletics             2       0     2
1      AFG  1936     Hockey            13       0    13
2      AFG  1948   Football            10       0    10
3      AFG  1948     Hockey            12       0    12
4      AFG  1956     Hockey            12       0    12
...    ...   ...        ...           ...     ...   ...
26224  ZIM  2020  

In [5]:
print(summary.columns)

Index(['NOC', 'Year', 'Sport', 'Participants', 'Female', 'Male', 'Bronze',
       'Gold', 'No medal', 'Silver'],
      dtype='object')


### 模型评估（未使用历史数据作为特征）

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [8]:
# 选择需要的特征列和目标列，TODO：添加上届参赛人员赛绩？
X = summary[['NOC', 'Year', 'Sport', 'Participants', 'Female', 'Male']]
y = summary['Gold']

# 对类别变量进行编码，例如NOC和Sport
X = pd.get_dummies(X, columns=['NOC', 'Sport'], drop_first=True)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 线性回归
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test, rf_y_pred)
print(f'Random Forest Mean Squared Error: {rf_mse}')

# 决策树
dt_model = DecisionTreeRegressor(max_depth=None, min_samples_split=5, max_features=None, max_leaf_nodes=None, random_state=42)
dt_model.fit(X_train, y_train)
dt_y_pred = dt_model.predict(X_test)

dt_mse = mean_squared_error(y_test, dt_y_pred)
print(f'Decision Tree Mean Squared Error: {dt_mse}')

# 梯度回归提升
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

# XBGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test, xgb_y_pred)
print("XGBoost Mean Squared Error:", xgb_mse)

Mean Squared Error: 331318.2568629713
Random Forest Mean Squared Error: 3.2816309950438423
Decision Tree Mean Squared Error: 4.400482378108188
Gradient Boosting Mean Squared Error: 3.6971943981621127
XGBoost Mean Squared Error: 3.575558223718012


In [9]:
# 同上，测银牌铜牌
print("Silver:")
X = summary[['NOC', 'Year', 'Sport', 'Participants', 'Female', 'Male']]
y = summary['Silver']

# 对类别变量进行编码，例如NOC和Sport
X = pd.get_dummies(X, columns=['NOC', 'Sport'], drop_first=True)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 线性回归
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test, rf_y_pred)
print(f'Random Forest Mean Squared Error: {rf_mse}')

# 决策树
dt_model = DecisionTreeRegressor(max_depth=None, min_samples_split=5, max_features=None, max_leaf_nodes=None, random_state=42)
dt_model.fit(X_train, y_train)
dt_y_pred = dt_model.predict(X_test)

dt_mse = mean_squared_error(y_test, dt_y_pred)
print(f'Decision Tree Mean Squared Error: {dt_mse}')

# 梯度回归提升
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

# XBGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test, xgb_y_pred)
print("XGBoost Mean Squared Error:", xgb_mse)

print("Bronze:")
X = summary[['NOC', 'Year', 'Sport', 'Participants', 'Female', 'Male']]
y = summary['Bronze']

# 对类别变量进行编码，例如NOC和Sport
X = pd.get_dummies(X, columns=['NOC', 'Sport'], drop_first=True)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 线性回归
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test, rf_y_pred)
print(f'Random Forest Mean Squared Error: {rf_mse}')

# 决策树
dt_model = DecisionTreeRegressor(max_depth=None, min_samples_split=5, max_features=None, max_leaf_nodes=None, random_state=42)
dt_model.fit(X_train, y_train)
dt_y_pred = dt_model.predict(X_test)

dt_mse = mean_squared_error(y_test, dt_y_pred)
print(f'Decision Tree Mean Squared Error: {dt_mse}')

# 梯度回归提升
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

# XBGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test, xgb_y_pred)
print("XGBoost Mean Squared Error:", xgb_mse)

Silver:
Mean Squared Error: 110964.27026578423
Random Forest Mean Squared Error: 4.136943537933663
Decision Tree Mean Squared Error: 5.7496002245096784
Gradient Boosting Mean Squared Error: 3.8312704506368247
XGBoost Mean Squared Error: 4.096944836475989
Bronze:
Mean Squared Error: 33345.69942984816
Random Forest Mean Squared Error: 3.8591373046130384
Decision Tree Mean Squared Error: 5.197629410767992
Gradient Boosting Mean Squared Error: 3.562157643785668
XGBoost Mean Squared Error: 3.888372831587583


In [10]:
# 假设延续24年的比赛项目：
sports = summary[summary['Year']==2024][['NOC','Year','Sport']]
sports['Year'] = 2025

X = summary[['NOC', 'Year', 'Sport', 'Participants', 'Female', 'Male']]
y = summary['Bronze']

# 对类别变量进行编码，例如NOC和Sport
X = pd.get_dummies(X, columns=['NOC', 'Sport'], drop_first=True)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

Gradient Boosting Mean Squared Error: 3.562157643785668


### 模型评估（使用历史数据） 以预测24年为测试

In [62]:
summary['Past_Gold'] = summary.groupby(['NOC','Sport'])['Gold'].shift(1)
summary['Past_Silver'] = summary.groupby(['NOC','Sport'])['Silver'].shift(1)
summary['Past_Bronze'] = summary.groupby(['NOC','Sport'])['Bronze'].shift(1)
summary['Past_Participants'] = summary.groupby(['NOC','Sport'])['Participants'].shift(1)
summary['Past_Female'] = summary.groupby(['NOC','Sport'])['Female'].shift(1)
summary['Past_Male'] = summary.groupby(['NOC','Sport'])['Male'].shift(1)
summary = summary[~pd.isna(summary['Past_Gold'])]
summary.to_csv("sports_summary.csv")

In [66]:
# 对 'NOC' 和 'Sport' 进行 one-hot 编码，并在 train 和 test 上分别应用相同的列
total_encoded = pd.get_dummies(summary, columns=['NOC', 'Sport'], drop_first=True)

train = total_encoded[total_encoded['Year'] != 2024]
test = total_encoded[total_encoded['Year'] == 2024]

# 获取训练集的特征列
X_train = train[['Year', 'Past_Gold', 'Past_Silver', 'Past_Bronze', 'Past_Participants', 'Past_Female', 'Past_Male']
                + [col for col in total_encoded.columns if col.startswith('NOC_') or col.startswith('Sport_')]]
# 获取训练集的标签（目标变量）
y_train1 = train['Gold']
y_train2 = train['Silver']
y_train3 = train['Bronze']
# 获取测试集的特征列
X_test = test[['Year', 'Past_Gold', 'Past_Silver', 'Past_Bronze', 'Past_Participants', 'Past_Female', 'Past_Male']
                + [col for col in total_encoded.columns if col.startswith('NOC_') or col.startswith('Sport_')]]
y_test1 = test['Gold']
y_test2 = test['Silver']
y_test3 = test['Bronze']

print("Gold")
# 线性回归
model = LinearRegression()
model.fit(X_train, y_train1)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test1, y_pred)
print(f"Mean Squared Error: {mse}")

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train1)
rf_y_pred = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test1, rf_y_pred)
print(f'Random Forest Mean Squared Error: {rf_mse}')

# 决策树
dt_model = DecisionTreeRegressor(max_depth=None, min_samples_split=5, max_features=None, max_leaf_nodes=None, random_state=42)
dt_model.fit(X_train, y_train1)
dt_y_pred = dt_model.predict(X_test)

dt_mse = mean_squared_error(y_test1, dt_y_pred)
print(f'Decision Tree Mean Squared Error: {dt_mse}')

# 梯度回归提升
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train1)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test1, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

# XBGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train1)
xgb_y_pred = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test1, xgb_y_pred)
print("XGBoost Mean Squared Error:", xgb_mse)

print("Silver")
# 线性回归
model = LinearRegression()
model.fit(X_train, y_train2)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test2, y_pred)
print(f"Mean Squared Error: {mse}")

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train2)
rf_y_pred = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test2, rf_y_pred)
print(f'Random Forest Mean Squared Error: {rf_mse}')

# 决策树
dt_model = DecisionTreeRegressor(max_depth=None, min_samples_split=5, max_features=None, max_leaf_nodes=None, random_state=42)
dt_model.fit(X_train, y_train2)
dt_y_pred = dt_model.predict(X_test)

dt_mse = mean_squared_error(y_test2, dt_y_pred)
print(f'Decision Tree Mean Squared Error: {dt_mse}')

# 梯度回归提升
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train2)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test2, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

# XBGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train2)
xgb_y_pred = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test2, xgb_y_pred)
print("XGBoost Mean Squared Error:", xgb_mse)

print("Bronze")
# 线性回归
model = LinearRegression()
model.fit(X_train, y_train3)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test3, y_pred)
print(f"Mean Squared Error: {mse}")

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train3)
rf_y_pred = rf_model.predict(X_test)

rf_mse = mean_squared_error(y_test3, rf_y_pred)
print(f'Random Forest Mean Squared Error: {rf_mse}')

# 决策树
dt_model = DecisionTreeRegressor(max_depth=None, min_samples_split=5, max_features=None, max_leaf_nodes=None, random_state=42)
dt_model.fit(X_train, y_train3)
dt_y_pred = dt_model.predict(X_test)

dt_mse = mean_squared_error(y_test3, dt_y_pred)
print(f'Decision Tree Mean Squared Error: {dt_mse}')

# 梯度回归提升
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train3)
gb_y_pred = gb_model.predict(X_test)

gb_mse = mean_squared_error(y_test3, gb_y_pred)
print("Gradient Boosting Mean Squared Error:", gb_mse)

# XBGBoost
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train3)
xgb_y_pred = xgb_model.predict(X_test)

xgb_mse = mean_squared_error(y_test3, xgb_y_pred)
print("XGBoost Mean Squared Error:", xgb_mse)

Gold


LinearRegression()

Mean Squared Error: 3.3005555117951397


RandomForestRegressor(random_state=42)

Random Forest Mean Squared Error: 3.858706336939722


...

Decision Tree Mean Squared Error: 6.269094538897476


GradientBoostingRegressor(random_state=42)

Gradient Boosting Mean Squared Error: 3.3326923315839525


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

XGBoost Mean Squared Error: 3.776687412869633


### 使用历史数据预测（使用随机森林）

In [12]:
summary['Past_Gold'] = summary.groupby(['NOC','Sport'])['Gold'].shift(1)
summary['Past_Silver'] = summary.groupby(['NOC','Sport'])['Silver'].shift(1)
summary['Past_Bronze'] = summary.groupby(['NOC','Sport'])['Bronze'].shift(1)
summary['Past_Participants'] = summary.groupby(['NOC','Sport'])['Participants'].shift(1)
summary['Past_Female'] = summary.groupby(['NOC','Sport'])['Female'].shift(1)
summary['Past_Male'] = summary.groupby(['NOC','Sport'])['Male'].shift(1)
summary = summary[~pd.isna(summary['Past_Gold'])]

In [13]:
summary.to_csv("sports_summary.csv")

In [14]:
test = summary[summary['Year']==2024]
test['Year'] = 2028
test['Participants'] = 0
test['Female'] = 0
test['Male'] = 0
test['Bronze'] = 0
test['Gold'] = 0
test['No medal'] = 0
test['Silver'] = 0

total = pd.concat([summary, test], axis=0, ignore_index=True)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14928\418316920.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Year'] = 2028
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14928\418316920.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Participants'] = 0
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14928\418316920.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [ ]:
# 对 'NOC' 和 'Sport' 进行 one-hot 编码，并在 train 和 test 上分别应用相同的列
total_encoded = pd.get_dummies(total, columns=['NOC', 'Sport'], drop_first=True)

train = total_encoded[total_encoded['Year'] != 2028]
test = total_encoded[total_encoded['Year'] == 2028]

# 获取训练集的特征列
X_train = train[['Year', 'Past_Gold', 'Past_Silver', 'Past_Bronze', 'Past_Participants', 'Past_Female', 'Past_Male']
                + [col for col in total_encoded.columns if col.startswith('NOC_') or col.startswith('Sport_')]]
# 获取训练集的标签（目标变量）
y_train1 = train['Gold']
y_train2 = train['Silver']
y_train3 = train['Bronze']
# 获取测试集的特征列
X_test = test[['Year', 'Past_Gold', 'Past_Silver', 'Past_Bronze', 'Past_Participants', 'Past_Female', 'Past_Male']
                + [col for col in total_encoded.columns if col.startswith('NOC_') or col.startswith('Sport_')]]

# 随机森林回归
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train1)
rf_y_pred1 = rf_model.predict(X_test)
print(rf_y_pred1)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train2)
rf_y_pred2 = rf_model.predict(X_test)
print(rf_y_pred2)
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train3)
rf_y_pred3 = rf_model.predict(X_test)
print(rf_y_pred3)

In [47]:
rf_y_pred1_df = pd.DataFrame(rf_y_pred1, columns=['Pred_Gold']).reset_index(drop=True)
rf_y_pred2_df = pd.DataFrame(rf_y_pred2, columns=['Pred_Silver']).reset_index(drop=True)
rf_y_pred3_df = pd.DataFrame(rf_y_pred3, columns=['Pred_Bronze']).reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_test_with_predictions = pd.concat([X_test, rf_y_pred1_df, rf_y_pred2_df, rf_y_pred3_df], axis=1)
X_test_with_predictions.to_csv("pred.csv")